# Код работы

## Загрузка модулей

In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import locale
import pymorphy3
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from copy import copy

import src.spellcheck
from src.dircheck import get_new_file_names
from src.processor import *

c:\Users\Artem Kondrashov\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:314: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


In [2]:
locale.setlocale(locale.LC_ALL, 'ru_RU')

'ru_RU'

## Общие настройки

In [ ]:
f_list = get_new_file_names("data/raw", "data/processed")
f_list

## Лист 1

In [ ]:
f_list[1]

In [ ]:
data_sheets = [pd.read_excel("data/raw/" + f_list[1], 
                             sheet_name=sheet_index) for sheet_index in range(4)]

In [ ]:
data_sheets[0].columns = ["Вопрос", "Ответ"]
data_sheets[0] = data_sheets[0][2:].dropna(subset = ["Вопрос"]).reset_index(drop = True)

In [ ]:
data_sheets_0_1 = data_sheets[0][:3].reset_index(drop = True)
data_sheets_0_1

In [ ]:
data_sheets_0_2 = data_sheets[0][3:].reset_index(drop = True)
data_sheets_0_2

### Тесты функций

Угадывание и исправление даты

In [ ]:
date_guesser_corrector("1966, 09 08")

## Лист 2

In [ ]:
data_sheets[1].columns = ["Месяц и год поступления", 
                          "Месяц и год увольнения",
                          "Должность с указанием наименования организации",
                          "Адрес организации"]

data_sheets[1] = data_sheets[1][2:].dropna().reset_index(drop = True)
data_sheets[1]

In [ ]:
datevec = data_sheets[1]["Месяц и год поступления"].to_numpy()
datevec

In [ ]:
datevec = data_sheets[1]["Месяц и год увольнения"].to_numpy()
datevec

In [ ]:
data_sheets[1]["Месяц и год поступления"].map(correct_date_condition2)

In [ ]:
data_sheets[1]["Месяц и год увольнения"].map(correct_date_condition2)

## Лист 3

In [ ]:
data_sheets[2].columns = ["Степень родства", 
                          "Фамилия, имя и отчество",
                          "Число, месяц, год и место рождения, гражданство",
                          "Место работы, должность",
                          "Адрес места жительства"]

data_sheets[2] = data_sheets[2][1:].dropna().reset_index(drop = True)
data_sheets[2]

In [ ]:
data_sheets[2]['Степень родства'].map(only_cyrillic)

! Для проверки условия 3.2 лучше использовать NER нейросеть.

In [ ]:
data_sheets[2]["Фамилия, имя и отчество"]

In [ ]:
data_sheets[2]["Фамилия, имя и отчество"].map(last_surnames)

In [ ]:
data_sheets[2]["Адрес места жительства"]

In [ ]:
text = data_sheets[2]["Адрес места жительства"][0]
text

In [ ]:
# adr = "г. Салават, ул. Калинина, д. 48, кв. 71, Республика Башкортостан"
adr = "гор. Хабаровск, ул. Ленина, д. 32, Хабаровский край"
sort_address(adr)

In [ ]:
data_sheets[2]["Адрес места жительства"].map(sort_address)

## Лист 4

In [ ]:
data_sheets[3]

In [ ]:
data_sheets_3_1 = data_sheets[3][2:5].dropna(how="all")

data_sheets_3_1.columns = ["Степень родства", 
                          "Фамилия, имя и отчество",
                          "Где проживает и период проживания за границей"]

data_sheets_3_1

In [ ]:
stop_idx = data_sheets[3][data_sheets[3].iloc[:, 0].str.contains("Дополнительные сведения", na=False)].index[0]
data_sheets_3_2 = data_sheets[3][7:stop_idx].dropna(how="all")

data_sheets_3_2.columns = ["Период проживания начало", 
                          "Период проживания конец",
                          "Адрес проживания и регистрации"]

data_sheets_3_2

# Запись в Excel

In [ ]:
pd_list = [data_sheets_0_1, data_sheets_0_2, data_sheets[1], data_sheets[2], data_sheets_3_1, data_sheets_3_2]

In [ ]:
data_sheets[2]

In [ ]:
lists[0][["Ответ"]]

In [ ]:
def save_to_excel(df_list: list, filename: str):

    template_file = "templates/form4.template.xlsx"
    output_file = "data/processed/" + filename.replace(".xlsx", "") + "_check.xlsx"

    wb = load_workbook(template_file)

    # Запись первого листа
    ws = wb['Лист1']

    # Часть ФИО
    rows = dataframe_to_rows(df_list[0][["Ответ"]], index=False, header=False)

    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx+3, column=c_idx+1, value=value)

    # Часть ответа на вопросы
    rows = dataframe_to_rows(df_list[1][["Ответ"]], index=False, header=False)

    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx+7, column=c_idx+1, value=value)

    # Запись второго листа
    ws = wb['Лист2']

    rows = dataframe_to_rows(df_list[2], index=False, header=False)

    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx+3, column=c_idx, value=value)

    # Запись третьего листа
    ws = wb['Лист3']

    rows = dataframe_to_rows(df_list[3], index=False, header=False)

    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx+2, column=c_idx, value=value)

    # Запись четвертого листа
    ws = wb['Лист4']

    # определение количества строк в п. 17 - добавление строк при необходимости
    if len(df_list[5]) > 11:
        rows_to_add2 = len(df_list[5]) - 11
        ws.insert_rows(20, rows_to_add2)
    else:
        rows_to_add2 = 0

    # определение количества строк в п. 16 - добавление строк при необходимости
    if len(df_list[4]) > 4:
        rows_to_add1 = len(df_list[4]) - 4
        ws.insert_rows(7, rows_to_add1)
    else:
        rows_to_add1 = 0

    # сохранение стиля ячейки
    cellstyle = {"fill": copy(ws.cell(4, 1).fill), 
                "font": copy(ws.cell(4, 1).font), 
                "number_format": copy(ws.cell(4, 1).number_format), 
                "border": copy(ws.cell(4, 1).border)}

    # запись п. 16
    rows = dataframe_to_rows(df_list[4], index=False, header=False)

    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):

            # print(r_idx)

            ws.cell(row=r_idx+3, column=c_idx).value = value
            ws.cell(row=r_idx+3, column=c_idx).fill = (cellstyle['fill'])
            ws.cell(row=r_idx+3, column=c_idx).font = (cellstyle['font'])
            ws.cell(row=r_idx+3, column=c_idx).number_format = (cellstyle['number_format'])
            ws.cell(row=r_idx+3, column=c_idx).border = (cellstyle['border'])

    # запись п. 17
    rows = dataframe_to_rows(df_list[5], index=False, header=False)

    for r_idx, row in enumerate(rows, 1):
        for c_idx, value in enumerate(row, 1):
            ws.cell(row=r_idx+9+rows_to_add1, column=c_idx).value = value
            ws.cell(row=r_idx+9+rows_to_add1, column=c_idx).fill = (cellstyle['fill'])
            ws.cell(row=r_idx+9+rows_to_add1, column=c_idx).font = (cellstyle['font'])
            ws.cell(row=r_idx+9+rows_to_add1, column=c_idx).number_format = (cellstyle['number_format'])
            ws.cell(row=r_idx+9+rows_to_add1, column=c_idx).border = (cellstyle['border'])

    wb.save(output_file)
    wb.close()



In [ ]:
f_list[1]

In [ ]:
save_to_excel(pd_list, f_list[1])

# Общий цикл

In [3]:
f_list = get_new_file_names("data/raw", "data/processed")
f_list

['Анкета_Макаров С.Н._18122023234522.xlsx',
 'Анкета_Алексеев О.Я._17012024235530.xlsx']

In [4]:
file_processor(f_list[1], verbose=True)

Обработка документа Анкета_Алексеев О.Я._17012024235530.xlsx
Анкета_Алексеев О.Я._17012024235530.xlsx: Листы считаны
Анкета_Алексеев О.Я._17012024235530.xlsx: Обработка первого листа
Проверка орфографии... Лист 1
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 1: орфография проверена
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 1: Условие 1 исправлено
Обработка листа 1 завершена
Анкета_Алексеев О.Я._17012024235530.xlsx: Обработка второго листа
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 2: Условие 2 исправлено
Проверка орфографии Лист 2...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 2: орфография проверена
Проверка условия 4


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 2: Условие 4 исправлено
Обработка листа 2 завершена
Анкета_Алексеев О.Я._17012024235530.xlsx: Обработка третьего листа
Проверка условия 3.1
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 3: Условие 3.1 исправлено
Проверка условия 3.2 (используется NER)
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 3: Условие 3.2 исправлено
Проверка орфографии Лист 3...
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 3: орфография проверена
Проверка условия 4 (тут нужен NER)
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 3: Условие 4 исправлено
Обработка листа 3 завершена
Анкета_Алексеев О.Я._17012024235530.xlsx: Обработка четвертого листа
Проверка условия 3.1
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 4: Условие 3.1 исправлено
Проверка условия 3.2
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 4: Условие 3.2 исправлено
Анкета_Алексеев О.Я._17012024235530.xlsx: Лист 4: Условие 2 исправлено
Проверка орфографии Лист 4...
Анкета_Алексеев О.Я._1701202423553